### Bronze Layer

#### Table Creation and Update

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS weather;
USE CATALOG weather;
CREATE SCHEMA IF NOT EXISTS 01_bronze;
USE SCHEMA 01_bronze;
CREATE TABLE IF NOT EXISTS weather_api_measurements_raw (
  -- Metadata
  file_path              STRING,
  ingestion_timestamp    TIMESTAMP,

  -- Location fields
  district               STRING,
  region                 STRING,
  country                STRING,
  lat                    DOUBLE,
  lon                    DOUBLE,
  tz_id                  STRING,
  localtime_epoch        BIGINT,
  localtime              STRING,

  -- Current weather fields
  last_updated_epoch     BIGINT,
  last_updated           STRING,
  temp_c                 DOUBLE,
  temp_f                 DOUBLE,
  is_day                 BIGINT,
  condition_text         STRING,
  condition_icon         STRING,
  condition_code         BIGINT,
  wind_mph               DOUBLE,
  wind_kph               DOUBLE,
  wind_degree            BIGINT,
  wind_dir               STRING,
  pressure_mb            DOUBLE,
  pressure_in            DOUBLE,
  precip_mm              DOUBLE,
  precip_in              DOUBLE,
  humidity               BIGINT,
  cloud                  BIGINT,
  feelslike_c            DOUBLE,
  feelslike_f            DOUBLE,
  windchill_c            DOUBLE,
  windchill_f            DOUBLE,
  heatindex_c            DOUBLE,
  heatindex_f            DOUBLE,
  dewpoint_c             DOUBLE,
  dewpoint_f             DOUBLE,
  vis_km                 DOUBLE,
  vis_miles              DOUBLE,
  uv                     DOUBLE,
  gust_mph               DOUBLE,
  gust_kph               DOUBLE,

  -- Air quality map values
  co                     DOUBLE,
  no2                    DOUBLE,
  o3                     DOUBLE,
  so2                    DOUBLE,
  pm2_5                  DOUBLE,
  pm10                   DOUBLE,
  us_epa_index           BIGINT,
  gb_defra_index         BIGINT
);

In [0]:
%sql
COPY INTO weather_api_measurements_raw
FROM (
  SELECT
    _metadata.file_path as file_path, 
    current_timestamp() as ingestion_timestamp,
    location.name as district,
    location.region as region,
    location.country as country,
    location.lat as lat,
    location.lon as lon,
    location.tz_id as tz_id,
    location.localtime_epoch as localtime_epoch,
    location.localtime as localtime,
    current.last_updated_epoch as last_updated_epoch,
    current.last_updated as last_updated,
    current.temp_c as temp_c,
    current.temp_f as temp_f,
    current.is_day as is_day,
    current.condition.text as condition_text,
    current.condition.icon as condition_icon,
    current.condition.code as condition_code,
    current.wind_mph as wind_mph,
    current.wind_kph as wind_kph,
    current.wind_degree as wind_degree,
    current.wind_dir as wind_dir,
    current.pressure_mb as pressure_mb,
    current.pressure_in as pressure_in,
    current.precip_mm as precip_mm,
    current.precip_in as precip_in,
    current.humidity as humidity,
    current.cloud as cloud,
    current.feelslike_c as feelslike_c,
    current.feelslike_f as feelslike_f,
    current.windchill_c as windchill_c,
    current.windchill_f as windchill_f,
    current.heatindex_c as heatindex_c,
    current.heatindex_f as heatindex_f,
    current.dewpoint_c as dewpoint_c,
    current.dewpoint_f as dewpoint_f,
    current.vis_km as vis_km,
    current.vis_miles as vis_miles,
    current.uv as uv,
    current.gust_mph as gust_mph,
    current.gust_kph as gust_kph,
    current.air_quality['co'] AS co,
    current.air_quality['no2'] AS no2,
    current.air_quality['o3'] AS o3,
    current.air_quality['so2'] AS so2,
    current.air_quality['pm2_5'] AS pm2_5,
    current.air_quality['pm10'] AS pm10,
    current.air_quality['us-epa-index'] AS us_epa_index,
    current.air_quality['gb-defra-index'] AS gb_defra_index
  FROM 's3://databricks-aws-sebastiancuya-bucket/weather-data/*/*/*/*/*/*' 
)
FILEFORMAT = JSON
FORMAT_OPTIONS('inferSchema' = 'true', 'ignoreCorruptFiles' = 'true');



In [0]:
%sql
OPTIMIZE weather_api_measurements_raw ZORDER BY ingestion_timestamp

In [0]:
%sql
    WITH table_history AS (
        SELECT * FROM (DESCRIBE HISTORY `weather`.`01_bronze`.`weather_api_measurements_raw`)
    )
    SELECT
        operationMetrics.numSkippedCorruptFiles
    FROM table_history
    WHERE operation = 'COPY INTO'
    LIMIT 1